# VQPE and UVQPE Utilities

## VQPE 

In [2]:
from __future__ import division
import sys,math,random,numpy as np,scipy,itertools,tqdm,warnings,matplotlib.animation,matplotlib.pylab as pl
from numpy import linalg as LA, inf
from scipy.linalg import svd,eig,eigh,toeplitz,circulant

def S_gen(ref_state, time_grid, E, eps=0, mu=0):
    """
    Generate overlap matrix elements S_{0,t} = <ɸ|exp(-iHt)|ɸ>.

    Parameters:
    - time_grid: General time grid (t_0 = 0, t_1, ..., t_{NT_max-1})
    - E: Energy spectrum of Hamiltonian (E_0, E_1, ..., E_{Hilbert space dimension})
    - eps: Standard deviation of gaussian noises on matrix elements
    - mu: Mean of gaussian noises on matrix elements (which is always set to zero)
    
    Returns:
    - S: Noisy overlap matrix elements [S_{0,t0} = 1, S_{0,t1}, ...]
    """
    NT_max = len(time_grid)
    S = np.zeros((2, NT_max))
    for j in range(NT_max):
        phases_j = np.exp(-1j * E * time_grid[j])
        overlap = np.vdot(ref_state, phases_j * ref_state)
        S[0,j], S[1,j] = overlap.real, overlap.imag
    S[:,1:] = S[:,1:] + np.random.normal(mu, eps, (2, NT_max-1)) 
    return S

def H_gen(ref_state, time_grid, E, eps=0, mu=0):
    
    """
    Generate Hamiltonian matrix elements H_{0,t} = <ɸ|H exp(-iHt)|ɸ>.

    Parameters:
    - time_grid: General time grid (t_0 = 0, t_1, ..., t_{NT_max-1})
    - E: Energy spectrum of Hamiltonian (E_0, E_1, ..., E_{Hilbert space dimension})
    - eps: Standard deviation of gaussian noises on matrix elements
    - mu: Mean of gaussian noises on matrix elements (which is always set to zero)
    
    Returns:
    - H: Noisy Hamiltonian matrix elements [H_{0,t0} = <ɸ|H|ɸ>, H_{0,t1}, ...]
    """
    NT_max = len(time_grid)
    H = np.zeros((2, NT_max))
    for j in range(NT_max):
        phases_j = np.exp(-1j * E * time_grid[j])
        overlap = np.vdot(ref_state, E * phases_j * ref_state)
        H[0,j], H[1,j] = overlap.real, overlap.imag
    H[0,0] = H[0,0] + np.random.normal(mu, eps)
    H[:,1:] = H[:,1:] + np.random.normal(mu, eps, (2, NT_max-1)) 
    return H

def General_Eig(t, H_matrix, S_matrix, s_SVD, r_SVD, eigindx):
    
    """
    Compute solution to the generalized eigenvalue problems of VQPE.
    
    Parameters:
    - t: current time step
    - H_matrix: Toeplitz matrix with entries 
                U_{jk} = <ɸ|exp(iHjdt)Hexp(-iHkdt)|ɸ>_{jk} + eps_{jk}
    - S_matrix: Toeplitz matrix with entries
                S_{jk} = <ɸ|exp(iHjdt)exp(-iHkdt)|ɸ>_{jk} + eps_{jk}
                where noise eps_{jk} is shared among matrix elements that have identical values
    - s_SVD: Absolute threshold value for SVD thresholding
    - r_SVD: Relative threshold value for SVD thresholding (0 < r_SVD < 1)
             If r_SVD > 0, then relative thresholding is employed
             otherwise we need to specify the value of s_SVD (0 < s_SVD)
    - eigindx: Index of the target eigenstate to be approximated
    
    Returns:
    - Approximate target eigenenergy (multiplied by unit timestep) estimated by VQPE
    """
    
    Ht, St = H_matrix[:t+1,:t+1], S_matrix[:t+1,:t+1]
    sval_S, rot = eigh(St)
    trunc = 0    
    if r_SVD > 0:
        sval_cut = r_SVD * np.max(sval_S)
    else:
        sval_cut = s_SVD           
        
    for j in range(St.shape[0]):
        if sval_S[j] > sval_cut:
            trunc = j                # singular value truncation 
            break
            
    Srot, Hrot = np.diag(sval_S[trunc:]), rot[:, trunc:].conj().T @ Ht @ rot[:, trunc:]
    eigval_H, eigvec_H = eigh(Hrot, Srot, eigvals_only=False) 
    return np.sort(eigval_H)[eigindx]

def VQPE_dmd(ref_state, time_grid, S_row_complex, H_row_complex, E, s_SVD, r_SVD, eigindx=0):
    
    """
    Run VQPE to estimate a target eigenenergy with SVD truncation.
    
    Parameters:
    - ref_state: 1d array of reference state represented in the Hamiltonian eigenbasis
    
    - time_grid[l] = l: 1d array of timegrid
    - S_row_complex: 1d array containing the first row of the noisy Toeplitz overlap matrix
                     (<ɸ|exp(-iHldt)|ɸ> of size NT_max where NT_max counts the total number of time steps)
    - H_row_complex: 1d array containing the first row of the noisy Toeplitz Hamiltonian matrix
                     (<ɸ|H exp(-iHldt)|ɸ> of size NT_max where NT_max counts the total number of time steps)
    - E: Energy spectrum of Hamiltonian (E_0, E_1, ..., E_{Hilbert space dimension})
    - s_SVD: Absolute threshold value for SVD thresholding
    - r_SVD: Relative threshold value for SVD thresholding (0 < r_SVD < 1)
             If r_SVD > 0, then relative thresholding is employed
             otherwise we need to specify the value of s_SVD (0 < s_SVD)
    - eigid: Index of the target eigenstate to be approximated
    
    Returns:
    - gs_E: 1D array containing approximate ground state energy evaluated over time_grid
    - S_matrix: full Toeplitz overlap matrix 
    - H_matrix: full Toeplitz Hamiltonian matrix
    
    *** Note: make sure to account for the timestep del_tau when computing the first rows of the matrices ***
    """
    
    NT_max = len(time_grid)
    gs_E = np.zeros(NT_max-eigindx) 
    S_matrix, H_matrix = toeplitz(S_row_complex).T, toeplitz(H_row_complex).T

    for j in range(eigindx, NT_max):
        gs_E[j-eigindx] = General_Eig(j, H_matrix, S_matrix, s_SVD, r_SVD, eigindx)
    return gs_E, S_matrix, H_matrix

## UVQPE 

In [3]:
def General_EigU(t, H_matrix, S_matrix, s_SVD, r_SVD, eigindx):
    
    """
    Compute solution to the generalized eigenvalue problems of multi-reference UVQPE.
    
    Parameters:
    - t: current time step
    - H_matrix: Toeplitz matrix with entries 
                U_{jk} = <ɸ|exp(iHjdt)exp(-iHdt)exp(-iHkdt)|ɸ>_{jk} + eps_{jk}
    - S_matrix: Toeplitz matrix with entries
                S_{jk} = <ɸ|exp(iHjdt)exp(-iHkdt)|ɸ>_{jk} + eps_{jk}
                where noise eps_{jk} is shared among matrix elements that have identical values
    - s_SVD: Absolute threshold value for SVD thresholding
    - r_SVD: Relative threshold value for SVD thresholding (0 < r_SVD < 1)
             If r_SVD > 0, then relative thresholding is employed
             otherwise we need to specify the value of s_SVD (0 < s_SVD)
    - eigindx: Index of the target eigenstate to be approximated
    
    Returns:
    - Approximate target eigenenergy (multiplied by unit timestep) estimated by UVQPE
    """

    Ht, St = H_matrix[:t+1,:t+1], S_matrix[:t+1,:t+1]
    sval_S, rot = eigh(St)
    trunc, s_S = 0, np.diag(sval_S)
    sval_cut = s_SVD   
    
    if r_SVD > 0:
        sval_cut = r_SVD * np.max(sval_S)
    for j in range(St.shape[0]):
        if sval_S[j] > sval_cut:
            trunc = j                # singular value truncation 
            break
            
    Srot, Hrot = rot[:, trunc:].conj().T @ St @ rot[:, trunc:], rot[:, trunc:].conj().T @ Ht @ rot[:, trunc:]
    eigval_H, eigvec_H = eig(Hrot, Srot) 
    eigarg_H = - np.angle(eigval_H)
    return np.sort(eigarg_H)[eigindx]

def UVQPE_dmd(ref_state, NT_max, del_tau, S_row_complex, E, s_SVD, r_SVD, eigindx=0):
    
     
    """
    Run UVQPE to estimate a target eigenenergy with SVD truncation.
    
    Parameters:
    - ref_state: 1d array of reference state represented in the Hamiltonian eigenbasis
    - NT_max: total number of time steps
    - del_tau: unit timestep 
      so that time_grid[l] = l*del_tau
    - S_row_complex: 1d array containing the first row of the noisy Toeplitz overlap matrix 
                     (<ɸ|exp(-iHl*del_tau)|ɸ> of size NT_max + 1 where NT_max counts the total number of time steps)
    - E: Energy spectrum of Hamiltonian (E_0, E_1, ..., E_{Hilbert space dimension})
    - s_SVD: Absolute threshold value for SVD thresholding
    - r_SVD: Relative threshold value for SVD thresholding (0 < r_SVD < 1)
             If r_SVD > 0, then relative thresholding is employed
             otherwise we need to specify the value of s_SVD (0 < s_SVD)
    - eigid: Index of the target eigenstate to be approximated
    
    Returns:
    - gs_E/del_tau: 1D array containing approximate ground state energy evaluated over time_grid
    - S_matrix: full Toeplitz overlap matrix 
    - H_matrix: full Toeplitz "Hamiltonian" or shifted overlap matrix (where the "Hamiltonian" here is exp(-iHdt))
    
    *** Note: make sure to account for the timestep del_tau when computing the first row of the overlap matrix ***
    """
    
    gs_E = np.zeros(NT_max-eigindx)
    S_matrix = toeplitz(S_row_complex[:NT_max]).T
    H_col_complex = np.zeros(NT_max, dtype=complex)
    H_col_complex[0], H_col_complex[1:] = S_row_complex[1], S_row_complex[:NT_max-1].conj()
    H_matrix = toeplitz(H_col_complex, S_row_complex[1:NT_max+1])
        
    for j in range(eigindx, NT_max):
        gs_E[j-eigindx] = General_EigU(j, H_matrix, S_matrix, s_SVD, r_SVD, eigindx)
    return gs_E/del_tau, S_matrix, H_matrix